### Understanding and implementing Vectorized Logistic Regression

This notebook will be an attempt to implement logistic regression to predict each applicant’s chance of admission based on their results on two exams. The data used is from Andrew Ng's ML course and what follows below is the course assignment completed in Python as opposed to Octave.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
%matplotlib inline

In [2]:
data = pd.read_csv('data/ex2data1.txt', header = None, names = ('Test1', 'Test2', 'Result'))
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 3 columns):
Test1     100 non-null float64
Test2     100 non-null float64
Result    100 non-null int64
dtypes: float64(2), int64(1)
memory usage: 2.4 KB


In [3]:
data.shape

(100, 3)

In [4]:
data.head()

,Test1,Test2,Result
0,34.623660,78.024693,0
1,30.286711,43.894998,0
2,35.847409,72.902198,0
3,60.182599,86.308552,1
4,79.032736,75.344376,1


As the hypothesis function takes the form hθ(x)=θ0+θ1x1+θ2x2+θ3x3+⋯+θnxn
With vectorization we will assume that input training data x0 will be 1. 

Also we will init the X, y and theta variables

In [5]:
#Training data - X and result - y
data.insert(0, 'Test0', 1)
cols = data.shape[1]
X = data.iloc[:,:cols-1]
y = data.iloc[:,cols-1:cols]

In [6]:
print(X.head())
print(y.head())

print(X.shape)
print(y.shape)

   Test0      Test1      Test2
0      1  34.623660  78.024693
1      1  30.286711  43.894998
2      1  35.847409  72.902198
3      1  60.182599  86.308552
4      1  79.032736  75.344376
   Result
0       0
1       0
2       0
3       1
4       1
(100, 3)
(100, 1)


Implement the sigmoid function

In [7]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

Now the cost/loss function - which is
cost(hθ(x), y) = 1/m * (-y.log(hθ(x))-(1-y).log(1-hθ(x)))

Before that some transformation

X - 118x3 theta - 3x1
theta (T) - 1x3

In [8]:
X = np.array(X.values)  
y = np.array(y.values)  
theta = np.zeros(3)

print(theta.shape)

X = np.matrix(X)
y = np.matrix(y)
theta = np.matrix(theta)
    
print(theta.shape)  

(3,)
(1, 3)


Hence the transformation as

X - 118x3 theta - 1x3

So we need a transpose so that theta(T) - 3x1

In [9]:
def cost(theta, X, y):
    X = np.matrix(X)
    y = np.matrix(y)
    theta = np.matrix(theta)
    
    #print("X in cost",X.shape)
    #print("theta in cost", theta.shape)
    prob1 = np.multiply(-y, np.log(sigmoid(X * theta.T)))
    prob2 = np.multiply((1-y), np.log(1-sigmoid(X * theta.T)))

    return np.sum(prob1-prob2)/(len(X))

In [10]:
cost(theta, X, y)

0.69314718055994529

Thats the cost with theta = all zeros

Now we need a gradient function to arrive at the best parameters with the least cost. 

In [11]:
parameters = int(theta.ravel().shape[1])
print(parameters)

3


A single gradient step - 
θ:=θ−α/mXT(g(Xθ)−y)

[  8.47457627e-03,   1.87880932e-02,   7.77711864e-05]

In [12]:
def gradient(theta, X, y):
    X = np.matrix(X)
    y = np.matrix(y)
    theta = np.matrix(theta)
    
    #Get the number of parameters
    parameters = int(theta.ravel().shape[1])
    grad = np.zeros(parameters)
    #print (X.shape)
    #print (theta.T.shape)
    error = sigmoid(X * theta.T) - y

    for i in range(parameters):
        term = np.multiply(error, X[:,i])
        grad[i] = np.sum(term) / len(X)

    return grad

In [13]:
def gradient_vec(theta, X, y):
    """
    Return gradient of theta 
    Args:
        theta is a 1D numpy array 
        X is a m x n numpy array w/ the first column being an intercept of 1
        y is a m x 1 numpy array, it is the label set for each obs in X
        
    """
    X = np.matrix(X)
    y = np.matrix(y)
    theta = np.matrix(theta)
    m = len(y)
    h_theta = sigmoid(X.dot(theta.T))
    
    grad = (h_theta - y.T).T.dot(X)/m
      
    
    return np.ndarray.flatten(grad)

Now the optimization 

In [14]:
def optimize(X, y, theta, num_iterations, learning_rate, print_cost = False):
    """
    This function optimizes w and b by running a gradient descent algorithm
    
    Arguments:
    w -- weights, a numpy array of size (n_x, 1)
    b -- bias, a scalar
    X -- data of shape (n_x, m)
    Y -- true "label" vector (containing 0 if class 1, 1 if class 2), of shape (1, m)
    num_iterations -- number of iterations of the optimization loop
    learning_rate -- learning rate of the gradient descent update rule
    print_cost -- True to print the loss every 100 steps
    
    Returns:
    params -- dictionary containing the weights w and bias b
    grads -- dictionary containing the gradients of the weights and bias with respect to the cost function
    costs -- list of all the costs computed during the optimization, this will be used to plot the learning curve.
    """
    costs = []
    
    for i in range(num_iterations):
        
        cost_i = cost(theta, X, y)
        grad = gradient(theta, X, y)
        
        
        theta = theta - learning_rate*grad

        if i % 100 == 0:
            costs.append(cost_i)
            
        if print_cost and i % 100 == 0:
            print(i)
            print ("Cost (iteration %i) = %f" %(i, cost_i))
            
    
        
    return theta, grad, costs

In [15]:
num_iterations = 6000
learning_rate = 0.0001
print_cost = True
parameters, grads, costs = optimize(X, y, theta, num_iterations, learning_rate, print_cost)
print(parameters)
print(grads)
print(costs)

0
Cost (iteration 0) = 0.693147
100
Cost (iteration 100) = 0.630171
200
Cost (iteration 200) = 0.629803
300
Cost (iteration 300) = 0.629691
400
Cost (iteration 400) = 0.629629
500
Cost (iteration 500) = 0.629578
600
Cost (iteration 600) = 0.629529
700
Cost (iteration 700) = 0.629480
800
Cost (iteration 800) = 0.629431
900
Cost (iteration 900) = 0.629383
1000
Cost (iteration 1000) = 0.629334
1100
Cost (iteration 1100) = 0.629285
1200
Cost (iteration 1200) = 0.629237
1300
Cost (iteration 1300) = 0.629188
1400
Cost (iteration 1400) = 0.629139
1500
Cost (iteration 1500) = 0.629091
1600
Cost (iteration 1600) = 0.629042
1700
Cost (iteration 1700) = 0.628993
1800
Cost (iteration 1800) = 0.628945
1900
Cost (iteration 1900) = 0.628896
2000
Cost (iteration 2000) = 0.628847
2100
Cost (iteration 2100) = 0.628799
2200
Cost (iteration 2200) = 0.628750
2300
Cost (iteration 2300) = 0.628702
2400
Cost (iteration 2400) = 0.628653
2500
Cost (iteration 2500) = 0.628605
2600
Cost (iteration 2600) = 0.62855

In [16]:
import scipy.optimize as opt
X = np.matrix(X)
y = np.matrix(y)
theta = np.zeros(3)  

result = opt.fmin_tnc(func=cost, x0=theta, fprime=gradient, args=(X, y))  
print ('\nCost at theta found by fmin_tnc:', cost(result[0], X, y))
print ('\nOptimal theta:',  result[0])



Cost at theta found by fmin_tnc: 0.203497701589

Optimal theta: [-25.16131863   0.20623159   0.20147149]


In [17]:
import scipy.optimize as opt
initial_theta = np.zeros(3)

myargs = (X, y)
opts = {'full_output': True, 'maxiter': 400}

optimal_theta, cost, grad_at_min, inv_hessian_matrix,\
fun_calls, grad_calls, warn_flags = opt.fmin_bfgs(cost,
                                initial_theta,
                                args=myargs,
                                fprime=gradient,
                                **opts)

print ('\nCost at theta found by fmin_bfgs:', cost)
print ('\noptimal theta:')
print (optimal_theta)

Optimization terminated successfully.
         Current function value: 0.203498
         Iterations: 23
         Function evaluations: 31
         Gradient evaluations: 31

Cost at theta found by fmin_bfgs: 0.20349770158944389

optimal theta:
[-25.16133284   0.2062317    0.2014716 ]


/Users/smritimohapatra/anaconda3/envs/root_clone/lib/python3.6/site-packages/ipykernel_launcher.py:9: RuntimeWarning: divide by zero encountered in log
  if __name__ == '__main__':
/Users/smritimohapatra/anaconda3/envs/root_clone/lib/python3.6/site-packages/ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in multiply
  if __name__ == '__main__':


In [18]:
def predict(theta, X):
    result = sigmoid (X * theta.T)
    return [1 if x >= 0.5 else 0 for x in result]

In [19]:
theta_min = np.matrix(result[0])  
predictions = predict(theta_min, X) 

In [20]:
print(predictions)

[0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1]


In [21]:
print(y)

[[0]
 [0]
 [0]
 [1]
 [1]
 [0]
 [1]
 [1]
 [1]
 [1]
 [0]
 [0]
 [1]
 [1]
 [0]
 [1]
 [1]
 [0]
 [1]
 [1]
 [0]
 [1]
 [0]
 [0]
 [1]
 [1]
 [1]
 [0]
 [0]
 [0]
 [1]
 [1]
 [0]
 [1]
 [0]
 [0]
 [0]
 [1]
 [0]
 [0]
 [1]
 [0]
 [1]
 [0]
 [0]
 [0]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [0]
 [0]
 [0]
 [1]
 [0]
 [1]
 [1]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [0]
 [1]
 [1]
 [0]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [0]
 [0]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [0]
 [1]
 [1]
 [0]
 [1]
 [1]
 [0]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]]


Create an accuracy function to test accuracy of predictions

### Logistic Regression with Regularization

Implementing a regularized version to help with non-linear relationships. We will start with data which we already know to not have a simple linear relationship

In [22]:
data = pd.read_csv('data/ex2data2.txt', header = None, names = ('Test1', 'Test2', 'Result'))
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 118 entries, 0 to 117
Data columns (total 3 columns):
Test1     118 non-null float64
Test2     118 non-null float64
Result    118 non-null int64
dtypes: float64(2), int64(1)
memory usage: 2.8 KB


In [23]:
data.head()

,Test1,Test2,Result
0,0.051267,0.69956,1
1,-0.092742,0.68494,1
2,-0.213710,0.69225,1
3,-0.375000,0.50219,1
4,-0.513250,0.46564,1


In [24]:
def new_features(data, deg=5):
    #Going upto 'deg' degrees
    x1 = data['Test1']  
    x2 = data['Test2']
    
    data.insert(3, 'Ones', 1)
    
    for i in range(1, deg):  
        for j in range(0, i):
            data['In' + str(i) + str(j)] = np.power(x1, i-j) * np.power(x2, j)
            
    ##Going to need to clean up the other inputs
    
    ##And as usual lets add 'Ones' to make up for x0 parameter
    
    data.drop('Test1', axis=1, inplace=True)
    data.drop('Test2', axis=1, inplace=True)
        
    print(data.head())

In [25]:
new_features(data, 5)

   Result  Ones      In10      In20      In21      In30      In31      In32  \
0       1     1  0.051267  0.002628  0.035864  0.000135  0.001839  0.025089   
1       1     1 -0.092742  0.008601 -0.063523 -0.000798  0.005891 -0.043509   
2       1     1 -0.213710  0.045672 -0.147941 -0.009761  0.031616 -0.102412   
3       1     1 -0.375000  0.140625 -0.188321 -0.052734  0.070620 -0.094573   
4       1     1 -0.513250  0.263426 -0.238990 -0.135203  0.122661 -0.111283   

       In40      In41      In42      In43  
0  0.000007  0.000094  0.001286  0.017551  
1  0.000074 -0.000546  0.004035 -0.029801  
2  0.002086 -0.006757  0.021886 -0.070895  
3  0.019775 -0.026483  0.035465 -0.047494  
4  0.069393 -0.062956  0.057116 -0.051818  


Now we need new cost and gradient functions which include a penalty parameter

In [26]:
def cost_reg(theta, X, y, penalty):
    X = np.matrix(X)
    y = np.matrix(y)
    theta = np.matrix(theta)
    
    #print("X in cost",X.shape)
    #print("theta in cost", theta.shape)
    prob1 = np.multiply(-y, np.log(sigmoid(X * theta.T)))
    prob2 = np.multiply((1-y), np.log(1-sigmoid(X * theta.T)))
    
    reg_term = (penalty / (2 * len(X))) * np.sum(np.power(theta[:,1:theta.shape[1]],2))

    return np.sum(prob1-prob2)/(len(X)) + reg_term

In [27]:
def gradient_reg(theta, X, y, penalty):
    X = np.matrix(X)
    y = np.matrix(y)
    theta = np.matrix(theta)
    
    #Get the number of parameters
    parameters = int(theta.ravel().shape[1])
    grad = np.zeros(parameters)
    #print (X.shape)
    #print (theta.T.shape)
    error = sigmoid(X * theta.T) - y

    for i in range(parameters):
        term = np.multiply(error, X[:,i])
    
        if (i == 0):
            grad[i] = np.sum(term) / len(X)
        else:
            grad[i] = (np.sum(term) / len(X)) + ((penalty / len(X)) * theta[:, i])

    return grad

In [28]:
#Prepare X, y and theta

##messed up data above
cols = data.shape[1]
X = data.iloc[:,1:cols]
y = data.iloc[:,0:1]

In [29]:
X = np.array(X.values)  
y = np.array(y.values)  
theta = np.zeros(11)

lamda = 1

cost_reg(theta, X, y, lamda)

0.6931471805599454

In [30]:
output = opt.fmin_tnc(func=cost_reg, x0=theta, fprime=gradient_reg, args=(X, y, lamda))  
output  

(array([ 0.53010249,  0.29075567, -1.60725763, -0.5821382 ,  0.01781027,
        -0.21329509, -0.40024142, -1.37144139,  0.02264303, -0.9503358 ,
         0.0344085 ]), 22, 1)

In [31]:
theta_min = np.matrix(output[0])  
predictions = predict(theta_min, X)
print(predictions)

[1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0]


#TODO - 
1. Add accuracy function
2. Plotting
3. Decision boundary plotting
4. Test data prediction
